# Heroes Of Pymoli Data Analysis

## Trend Analysis

- Of the 576 players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).
- Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).
- Oathbreaker, Last Hope of the Breaking Storm is the most profitable and popular item around the players. The item was purchased 12 times making total revenue of $ 50.76.

## Pre-requisites

### 1. Dependencies

In [2]:
import pandas as pd

### 2. Loading Dataframes objects

In [3]:
# reading csv file and using Purchase ID as the Dataframe index
# pymoli_unique is a Dataframe that store non-duplicate values

pymoli = pd.read_csv('resources/purchase_data.csv', index_col='Purchase ID')
pymoli_unique = pymoli.groupby(by='SN', as_index = False).first()

pymoli.head(5)

,SN,Age,Gender,Item ID,Item Name,Price
Purchase ID,,,,,,
0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,Ithergue48,24,Male,92,Final Critic,4.88
3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,Iskosia90,23,Male,131,Fury,1.44


### 3. Dataframe validation (cheking for errors, null-values, etc)

In [103]:
# checking for null values

pymoli.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 780 entries, 0 to 779
Data columns (total 6 columns):
SN           780 non-null object
Age          780 non-null int64
Gender       780 non-null object
Item ID      780 non-null int64
Item Name    780 non-null object
Price        780 non-null float64
dtypes: float64(1), int64(2), object(3)
memory usage: 42.7+ KB


## Data Analysis

### 1. Player Count

In [8]:
total_players = pymoli_unique['SN'].count()

pd.DataFrame({"Total Players":[total_players]})

,Total Players
0,576


### 2. Purchasing Analysis (Total)

In [9]:
unique_items = pymoli['Item ID'].nunique()
average_price = pymoli['Price'].mean()
number_purchases = len(pymoli.index.values)
total_revenue = pymoli['Price'].sum()

purchasing_data = [{ "Number of Unique Items" : unique_items, 
                     "Average Price" : average_price, 
                     "Number of Purchases" : number_purchases, 
                     "Total Revenue" : total_revenue}]

purchasing_df = pd.DataFrame(data=purchasing_data, columns=['Number of Unique Items', 'Average Price', 'Number of Purchases', 'Total Revenue'])

purchasing_df['Average Price'] = purchasing_df['Average Price'].map("${:,.2f}".format)
purchasing_df['Total Revenue'] = purchasing_df['Total Revenue'].map("${:,.2f}".format)

purchasing_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,"$2,379.77"


### 3. Gender Demographics

In [10]:
gender_df = pd.DataFrame(pymoli_unique['Gender'].value_counts())
gender_df.rename(columns = {'Gender': 'Total Count'}, inplace=True)
gender_df['Percentage of Players'] = ((gender_df['Total Count'] / total_players) * 100).map("{:.2f}".format)

gender_df

,Total Count,Percentage of Players
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


### 4. Purchasing Analysis (Gender)

In [19]:
gender_purchasing_group = pymoli.groupby(by='Gender')
gender_purchasing_df = gender_purchasing_group.agg({'Price' : [('Purchase Count', 'count'), 
                                                               ('Average Purchase Price', 'mean'), 
                                                               ('Total Purchase Value', 'sum')]})['Price']

gender_purchasing_df['Avg Total Purchase per Person'] = gender_purchasing_df['Total Purchase Value'] / gender_df['Total Count']

gender_purchasing_df.style.format({'Average Purchase Price' : '${:,.2f}',
                                   'Total Purchase Value' : '${:,.2f}',
                                   'Avg Total Purchase per Person' : '${:,.2f}'})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


### 5. Age Demographics

In [108]:
# The criteria to bin by.
# int : Defines the number of equal-width bins in the range of x. The range of x is extended by .1% on each side to include 
# the minimum and maximum values of x.

bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 150.9]
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

age_demo_df = pd.DataFrame({'Age' : pymoli_unique['Age']})
age_demo_df['age_buckets'] = pd.cut(age_demo_df['Age'], bins, labels=group_names)

age_demo_grp = age_demo_df.groupby(by='age_buckets')
age_demo_grp_df = age_demo_grp.count()

age_demo_grp_df['Percentage of Players'] = ((age_demo_grp_df['Age']/age_demo_df['age_buckets'].count()) * 100).map("{:.2f}".format)

age_demo_grp_df.rename(columns = {'Age': 'Total Count' }, inplace=True )
age_demo_grp_df.index.name = None

age_demo_grp_df

,Total Count,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


### 6. Purchasing Analysis (Age)

In [22]:
bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 150.9]
group_names = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

purchasing_analysis = pd.DataFrame({'Price' : pymoli['Price'], 'Age' : pymoli['Age'], 'Gender' : pymoli['Gender']})
purchasing_analysis['age_buckets'] = pd.cut(purchasing_analysis['Age'], bins, labels=group_names)

purchasing_analysis_unique = pd.DataFrame({'Age' : pymoli_unique['Age']})
purchasing_analysis_unique['age_buckets'] = pd.cut(purchasing_analysis_unique['Age'], bins, labels=group_names)

group_analysis = purchasing_analysis.groupby(by='age_buckets')
pivot_analysis = group_analysis.agg({'Price' : [('Purchase Count', 'count'), 
                                                ('Average Purchase Price', 'mean'), 
                                                ('Total Purchase Value', 'sum')]})['Price']

pivot_analysis['Avg Total Purchase per Person'] = (pivot_analysis['Total Purchase Value'] / purchasing_analysis_unique.groupby(by='age_buckets').count()['Age'])

pivot_analysis['Average Purchase Price'] = pivot_analysis['Average Purchase Price'].map('${:,.2f}'.format)
pivot_analysis['Total Purchase Value'] = pivot_analysis['Total Purchase Value'].map('${:,.2f}'.format)
pivot_analysis['Avg Total Purchase per Person'] = pivot_analysis['Avg Total Purchase per Person'].map('${:,.2f}'.format)

pivot_analysis.index.name = None
pivot_analysis

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


### 7. Top Spenders

In [27]:
top_spenders = pd.DataFrame({'SN' : pymoli['SN'], 'Price' : pymoli['Price']})

group_spenders = top_spenders.groupby(by='SN')
pivot_spenders = group_spenders.agg({'Price' : [('Purchase Count', 'count'), 
                                                ('Average Purchase Price', 'mean'), 
                                                ('Total Purchase Value', 'sum')]})['Price']

pivot_spenders.sort_values(by=['Total Purchase Value'], ascending=False, inplace=True)

pivot_spenders['Average Purchase Price'] = pivot_spenders['Average Purchase Price'].map('${:,.2f}'.format)
pivot_spenders['Total Purchase Value'] = pivot_spenders['Total Purchase Value'].map('${:,.2f}'.format)

pivot_spenders.head(5)

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


### 8. Most Popular Items

In [33]:
popular_items = pd.DataFrame({'Item ID' : pymoli['Item ID'], 'Item Name' : pymoli['Item Name'], 'Price' : pymoli['Price']})
popular_items_grp = popular_items.groupby(by=['Item ID', 'Item Name'])

pivoti_popular_items = popular_items_grp.agg({'Price' : [('Purchase Count', 'count'), 
                                                         ('Item Price', lambda value: (value.sum()/value.count())), 
                                                         ('Total Purchase Value', 'sum')]})['Price']

pivoti_popular_items.sort_values(by=['Purchase Count'], ascending=False, inplace=True)

pivoti_popular_items['Item Price'] = pivoti_popular_items['Item Price'].map('${:,.2f}'.format)
pivoti_popular_items['Total Purchase Value'] = pivoti_popular_items['Total Purchase Value'].map('${:,.2f}'.format)

pivoti_popular_items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


### 9. Most Profitable Items

In [47]:
pivoti_popular_items['Item Price'] = pivoti_popular_items['Item Price'].replace(to_replace='\$', value='', regex=True).astype(float)
pivoti_popular_items['Total Purchase Value'] = pivoti_popular_items['Total Purchase Value'].replace(to_replace='\$', value='', regex=True).astype(float)

pivoti_popular_items.sort_values(by=['Total Purchase Value'], ascending=False, inplace=True)

pivoti_popular_items['Item Price'] = pivoti_popular_items['Item Price'].map('${:,.2f}'.format)
pivoti_popular_items['Total Purchase Value'] = pivoti_popular_items['Total Purchase Value'].map('${:,.2f}'.format)

pivoti_popular_items.head(5)

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
